# MDL DB read

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['lines.linewidth'] = 1.5

import os
import copy

import data
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Run
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.analytical_linear_estimator import AnalyticalLinearEstimator
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator

from mdldb import mdl_to_evaluation
from evaluation.run_dynamic import RunDynamic
from evaluation.run_manoeuvring import RunZigZag
from sklearn.pipeline import Pipeline
import signal_lab

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

In [ ]:
db.session.query(Run.test_type).distinct().all()

In [ ]:
len(db.session.query(Run).filter(Run.test_type=='roll decay').all())

In [ ]:
roll_decay_tests = db.session.query(Run).filter(Run.test_type=='roll decay')

In [ ]:
db_run = roll_decay_tests[10]

In [ ]:
db_run.ship_speed

In [ ]:
db_run.loading_condition.name

In [ ]:
ascii_file = db_run.load()

In [ ]:
run = mdl_to_evaluation.db_run_to_evaluation_run(run=db_run)

In [ ]:
run.track_plot()

In [ ]:
run.plot('roll')

In [ ]:
ascii_file = db_run.load()
df_raw = ascii_file.channels
df = signal_lab.mdl_to_evaluation.do_transforms(df=df_raw)
df.rename(columns={'MA/Roll':'phi'}, inplace=True)
df.plot(y='phi')

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=1, minimum_score=0.99)
scaler = ScaleFactorTransformer(scale_factor=db_run.model.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(1))
offset_transformer = OffsetTransformer()


In [ ]:
df_cut = cutter.fit_transform(df)
df_cut.plot(y='phi')

In [ ]:
analytical_estimator = AnalyticalLinearEstimator()

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
    #('offset_transformer',offset_transformer),
    ('analytical_linear_estimator', analytical_estimator)]
        
pipeline_analytical_linear = Pipeline(steps) # define the pipeline object.

In [ ]:
def fit_predict(db_run, pipeline):
    
    ascii_file = db_run.load()
    df_raw = ascii_file.channels
    
    df = signal_lab.mdl_to_evaluation.do_transforms(df=df_raw)
        
    _pipline = copy.deepcopy(pipeline)
    
    scaler = _pipline['scaler']
    scaler.scale_factor = db_run.model.scale_factor
    
    df.rename(columns={'MA/Roll':'phi'}, inplace=True)
    
    # Fit:
    _pipline.fit(X=df)
    
    # Predict:
    estimator = _pipline[-1]
    s = {} 
    s.update(estimator.parameters)
    s['score'] = estimator.score(X=estimator.X)
    s['mean_damping'] = estimator.calculate_average_linear_damping()
    X = estimator.X
    s['phi_start'] = X.iloc[0]['phi']
    s['phi_stop'] = X.iloc[-1]['phi']
    
    
    if not 'd' in s:
        s['d'] = 0
    
    return s,_pipline, df

In [ ]:
s,_pipline, df = fit_predict(db_run=db_run, pipeline=pipeline_analytical_linear)

In [ ]:
s

In [ ]:
estimator = _pipline[-1]
estimator.plot_fit()

In [ ]:
ascii_file = db_run.load()
df_raw = ascii_file.channels

df = signal_lab.mdl_to_evaluation.do_transforms(df=df_raw)
df.rename(columns={'MA/Roll':'phi'}, inplace=True)

In [ ]:
np.rad2deg(df['phi'].max())

In [ ]:
ftol = 10**-15

phi_maxs = np.linspace(9,2,10)
df_result = pd.DataFrame()

for phi_max in phi_maxs:
    
    lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=1, minimum_score=0.99)
    cutter = CutTransformer(phi_max=np.deg2rad(phi_max), phi_min=np.deg2rad(0.25))
    
    steps = [('filter',lowpass_filter),
             ('scaler',scaler),
             ('cutter', cutter),
            ]
    
    preprocess = Pipeline(steps)
    X = preprocess.fit_transform(df)
    
    analytical_estimator = AnalyticalLinearEstimator(ftol=ftol, omega_regression=True)
    
    analytical_estimator.fit(X=X)
    
    fig,axes=plt.subplots(ncols=2)
    fig.set_size_inches(14,6)
    analytical_estimator.plot_fit(ax=axes[0])
    analytical_estimator.plot_fft(ax=axes[1])
    axes[1].set_xlim(0,0.7)
    
    
    # Predict:
    estimator = analytical_estimator
    s={}
    s.update(estimator.parameters)
    s['score'] = estimator.score(X=estimator.X)
    s['mean_damping'] = estimator.calculate_average_linear_damping()
    X = estimator.X
    s['phi_start'] = X.iloc[0]['phi']
    s['phi_stop'] = X.iloc[-1]['phi']
    
    s = pd.Series(s, name=phi_max)
    df_result=df_result.append(s)


In [ ]:
fig,ax = plt.subplots()
df_result.plot(y='score', ax=ax, label='score')
ax.legend(loc='upper left')
ax.set_ylabel('score')
ax.set_xlabel('$max(\phi)$ [deg]')

ax2 = ax.twinx()
ax2.set_ylabel('omega0', color='red')
df_result.plot(y='omega0', ax=ax2, style='r-')

fig,ax = plt.subplots()
df_result.plot(y='zeta', ax=ax)



In [ ]:
analytical_estimator.X_amplitudes.plot(x='phi',y = 'omega0', style='o')

In [ ]:
omega0_max = analytical_estimator.X_amplitudes['omega0'].max()
omega0_min = analytical_estimator.X_amplitudes['omega0'].min()

df = X.copy()
df['phi_omega_max'] = analytical_estimator._equation_omega(df=X,omega0=omega0_max,zeta = analytical_estimator.parameters['zeta'])
df['phi_omega_min'] = analytical_estimator._equation_omega(df=X,omega0=omega0_min,zeta = analytical_estimator.parameters['zeta'])

fig,ax=plt.subplots()
fig.set_size_inches(14,7)
fig.set_dpi(200)
df.plot(y='phi', label='model test', ax=ax)
df.plot(y='phi_omega_max', ax=ax)
df.plot(y='phi_omega_min', ax=ax)
ax.legend()




In [ ]:
ftol = 10**-15

phi_maxs = np.linspace(4,2,10)
df_result = pd.DataFrame()

for phi_max in phi_maxs:
    
    lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=1, minimum_score=0.99)
    cutter = CutTransformer(phi_max=np.deg2rad(phi_max), phi_min=np.deg2rad(0.25))
    
    steps = [('filter',lowpass_filter),
             ('scaler',scaler),
             ('cutter', cutter),
            ]
    
    preprocess = Pipeline(steps)
    X = preprocess.fit_transform(df)
    
    analytical_estimator = DirectLinearEstimator(ftol=ftol, omega_regression=False)
    
    analytical_estimator.fit(X=X)
    
    fig,axes=plt.subplots(ncols=2)
    fig.set_size_inches(14,6)
    analytical_estimator.plot_fit(ax=axes[0])
    analytical_estimator.plot_fft(ax=axes[1])
    axes[1].set_xlim(0,0.7)

    
    
    # Predict:
    estimator = analytical_estimator
    s={}
    s.update(estimator.parameters)
    s['score'] = estimator.score(X=estimator.X)
    s['mean_damping'] = estimator.calculate_average_linear_damping()
    X = estimator.X
    s['phi_start'] = X.iloc[0]['phi']
    s['phi_stop'] = X.iloc[-1]['phi']
    
    s = pd.Series(s, name=phi_max)
    df_result=df_result.append(s)

In [ ]:
analytical_estimator.X_amplitudes.plot(x='phi',y = 'omega0', style='o')

In [ ]:
fig,ax = plt.subplots()
df_result.plot(y='score', ax=ax, label='score')
ax.legend(loc='upper left')
ax.set_ylabel('score')
ax.set_xlabel('$max(\phi)$ [deg]')

ax2 = ax.twinx()
ax2.set_ylabel('omega0', color='red')
df_result.plot(y='omega0', ax=ax2, style='r-')

fig,ax = plt.subplots()
df_result.plot(y='zeta', ax=ax)

In [ ]:
X.plot(y='phi')

In [ ]:
steps = [('filter',lowpass_filter),
         ('scaler',scaler),
         ('cutter', cutter),
        ]

preprocess = Pipeline(steps)
X = preprocess.fit_transform(df)

linear_estimator = DirectLinearEstimator(ftol = ftol)
linear_estimator.fit(X=X)
linear_estimator.plot_fit()

In [ ]:
linear_estimator

In [ ]:
linear_estimator.omega0

In [ ]:
linear_estimator.score()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,5)
fig.set_dpi(200)
X.plot(y='phi', ax=ax, label='model test', alpha=0.5)
linear_estimator.predict(X).plot(y='phi', ax=ax, label='numerical', style='-')
analytical_estimator.predict(X).plot(y='phi', ax=ax, style='--', label='analytical')

fig,ax=plt.subplots()
fig.set_size_inches(10,5)
fig.set_dpi(200)
X.plot(y='phi1d', ax=ax, label='model test', alpha=0.5)
linear_estimator.predict(X).plot(y='phi1d', ax=ax, label='numerical', style='-', )
analytical_estimator.predict(X).plot(y='phi1d', ax=ax, style='--', label='analytical')

fig,ax=plt.subplots()
fig.set_size_inches(10,5)
fig.set_dpi(200)
X.plot(y='phi2d', ax=ax, label='model test', alpha=0.5)
linear_estimator.predict(X).plot(y='phi2d', ax=ax, label='numerical', style='-')
analytical_estimator.predict(X).plot(y='phi2d', ax=ax, style='--', label='analytical')


In [ ]:
from rolldecay import database
df_rolldecay_analytical = database.load(rolldecay_table_name='rolldecay_linear_analytical', limit_score=0.6)
df_rolldecay_linear = database.load(rolldecay_table_name='rolldecay_linear', limit_score=0.5)
index = list(set(df_rolldecay_analytical.index) & set(df_rolldecay_linear.index))
df_rolldecay_analytical = df_rolldecay_analytical.loc[index]
df_rolldecay_linear = df_rolldecay_linear.loc[index]


In [ ]:
fig,ax = plt.subplots()
x = df_rolldecay_linear['omega0']
y = df_rolldecay_analytical['omega0']
ax.plot(x,y,'o', alpha=0.5)


In [ ]:
fig,ax = plt.subplots()
x = df_rolldecay_linear['zeta']
y = df_rolldecay_analytical['zeta']
ax.plot(x,y,'o', alpha=0.5)

In [ ]:
fig,ax = plt.subplots()
x = df_rolldecay_linear['score']
y = df_rolldecay_analytical['score']
ax.plot(x,y,'o', alpha=0.5)

In [ ]:
df_worst = df_rolldecay_linear.sort_values(by= ['score']).iloc[0:10].copy()

In [ ]:
#for run_id,row in df_worst.iterrows():
    
    
    